In [ ]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Change this certs path to wherever you're storing your certificates

# Your credentials.json file should look like this:

# {
#     "username" : "johnsmith123",
#     "password" : "guest",
#     "app_key" : "****************"
# }

In [ ]:
credentials_file_path = '/credentials.json'
certificates_file_path = '/certs'

In [ ]:
with open(credentials_file_path) as f:
    cred = json.load(f)
    my_username = cred['username']
    my_password = cred['password']
    my_app_key = cred['app_key']

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certificates_file_path,
                                       locale = "australia",
                                       lightweight=True)

trading.login()

In [ ]:
# Grab all event type ids. This will return a list which we will iterate over to print out the id and the name of the sport
event_types = trading.betting.list_event_types()
event_types[0]

In [ ]:
sport_ids = pd.DataFrame({
    'Sport': [event_type_object['eventType']['name'] for event_type_object in event_types],
    'ID': [event_type_object['eventType']['id'] for event_type_object in event_types]
}).set_index('Sport').sort_index()

sport_ids

In [ ]:
# Filter for just AFL
afl_filter = betfairlightweight.filters.market_filter(text_query='Australian Rules')
afl_filter

In [ ]:
# This returns a list
afl_event_type = trading.betting.list_event_types(
    filter=afl_filter)
afl_event_type

In [ ]:
# Get the first element of the list
afl_event_type = afl_event_type[0]
afl_event_type

In [ ]:
afl_event_type_id = afl_event_type['eventType']['id']
print(f"The event type id for AFL is {afl_event_type_id}")

# The event type id for AFL is 61420

In [ ]:
# Get a datetime object in a week and convert to string
datetime_in_a_week = (datetime.datetime.utcnow() + datetime.timedelta(weeks=1)).strftime("%Y-%m-%dT%TZ")

# Create a competition filter
competition_filter = betfairlightweight.filters.market_filter(
    event_type_ids=[61420],
    market_start_time={
        'to': datetime_in_a_week
    })

competition_filter

In [ ]:
# Get a list of competitions for AFL
competitions = trading.betting.list_competitions(
    filter=competition_filter
)
competitions

In [ ]:
# Iterate over the competitions and create a dataframe of competitions and competition ids
afl_competitions = pd.DataFrame({
    'Competition': [competition_object['competition']['name'] for competition_object in competitions],
    'ID': [competition_object['competition']['id'] for competition_object in competitions]
})
afl_competitions

In [ ]:
# Define a market filter
afl_event_filter = betfairlightweight.filters.market_filter(
    event_type_ids=[afl_event_type_id],
    market_countries=['AU'],
    market_start_time={
        'to': (datetime.datetime.utcnow() + datetime.timedelta(days=7)).strftime("%Y-%m-%dT%TZ")
    }
)

# Print the filter
afl_event_filter

In [ ]:
# Get a list of all thoroughbred events as objects
aus_afl_events = trading.betting.list_events(
    filter=afl_event_filter
)
aus_afl_events[3]

In [ ]:
# Create a DataFrame with all the events by iterating over each event object
aus_afl_events_today = pd.DataFrame({
    'Event Name': [event_object['event']['name'] for event_object in aus_afl_events],
    'Event ID': [event_object['event']['id'] for event_object in aus_afl_events],
    # 'Event Venue': [event_object['event']['venue'] for event_object in aus_afl_events],
    'Country Code': [event_object['event']['countryCode'] for event_object in aus_afl_events],
    'Time Zone': [event_object['event']['timezone'] for event_object in aus_afl_events],
    'Open Date': [event_object['event']['openDate'] for event_object in aus_afl_events],
    'Market Count': [event_object['marketCount'] for event_object in aus_afl_events]
})

aus_afl_events_today

In [ ]:
event_name = 'Richmond v Gold Coast'
event_id = aus_afl_events_today.loc[aus_afl_events_today['Event Name'] == event_name, 'Event ID'].iloc[0]
event_id

In [ ]:
# Define a market filter
market_types_filter = betfairlightweight.filters.market_filter(
    event_ids=[event_id]
)
market_types_filter

In [ ]:
# Request market types
market_types = trading.betting.list_market_types(
        filter=market_types_filter
)
market_types

In [ ]:
# Create a DataFrame of market types
market_types_event = pd.DataFrame({
    'Market Type': [market_type_object['marketType'] for market_type_object in market_types],
})

market_types_event

In [ ]:
market_catalogue_filter = betfairlightweight.filters.market_filter(
    event_ids=[event_id]
)
market_catalogue_filter

In [ ]:
market_catalogues = trading.betting.list_market_catalogue(
    filter=market_catalogue_filter,
    max_results='100',
    sort='FIRST_TO_START'
)
market_catalogues[3]

In [ ]:
# Create a DataFrame for each market catalogue
market_types_event = pd.DataFrame({
    'Market Name': [market_cat_object['marketName'] for market_cat_object in market_catalogues],
    'Market ID': [market_cat_object['marketId'] for market_cat_object in market_catalogues],
    'Total Matched': [market_cat_object['totalMatched'] for market_cat_object in market_catalogues],
})

market_types_event

In [ ]:
# Create a price filter. Get all traded and offer data
price_filter = betfairlightweight.filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)
price_filter

In [ ]:
market_id = market_types_event[market_types_event['Market Name'] == "Total Points"]['Market ID'].iloc[0]
market_id

In [ ]:
# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[market_id],
    price_projection=price_filter
)
# market_books÷

In [ ]:
# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]
# market_book

In [ ]:
traded_markets = [market for market in market_book['runners'] if 'lastPriceTraded' in list(market.keys())]

In [ ]:
traded_markets

In [ ]:
def process_event_books(event_books):
    '''
    This function processes the geelong books and returns a DataFrame with the best back/lay prices + vol for each geelong
    :param event_books:
    :return:
    '''

    selection_ids = [event_book['selectionId'] for event_book in event_books]
    total_points_line = [event_book['handicap'] for event_book in event_books]
    last_prices_traded = [event_book['lastPriceTraded'] for event_book in event_books]
    total_matched = [event_book['totalMatched'] for event_book in event_books]

    df = pd.DataFrame({
        'Selection ID': selection_ids,
        'Total Points Line': total_points_line,
        'Last Price Traded': last_prices_traded,
        'Total Matched': total_matched,
    })

    df['Selection Name'] = np.where(df['Selection ID'] == 8435232, 'Under', 'Over')
    
    df = df[['Selection ID', 'Selection Name', 'Total Points Line', 'Last Price Traded', 'Total Matched']]
    
    return df


In [ ]:
event_df = process_event_books(traded_markets)
event_df

In [ ]:
# Get the favourite's price and selection id
best_selection_id = event_df.loc[event_df['Last Price Traded'].idxmax(), 'Selection ID']
best_price = event_df.loc[event_df['Last Price Traded'].idxmax(), 'Last Price Traded']
best_selection_id, best_price

In [ ]:
# Define a limit order filter
limit_order_filter = betfairlightweight.filters.limit_order(
    size=5, 
    price=7,
    persistence_type='LAPSE'
)
limit_order_filter

In [ ]:
# Define an instructions filter
instructions_filter = betfairlightweight.filters.place_instruction(
    selection_id=str(8435232),
    order_type="LIMIT",
    side="BACK",
    limit_order=limit_order_filter
)
instructions_filter


In [ ]:
# Place the order
order = trading.betting.place_orders(
    market_id=market_id, # The market id we obtained from before
    customer_strategy_ref='tp_model',
    instructions=[instructions_filter] # This must be a list
)


In [ ]:
order

In [ ]:
trading.betting.list_current_orders(
    customer_strategy_refs=['tp_model']
)

In [ ]:
cancelled_order = trading.betting.cancel_orders(market_id=market_id)
cancelled_order

In [ ]:
# # Create a DataFrame to view the instruction report
# pd.Series(cancelled_order.cancel_instruction_reports[0].__dict__).to_frame().T

In [ ]:
# listClearedOrders
cleared_orders = trading.betting.list_cleared_orders(bet_status="SETTLED",
                                                    market_ids=[market_id])
cleared_orders

In [ ]:
# Create a DataFrame from the orders
# pd.DataFrame(cleared_orders._data['clearedOrders'])

In [ ]:
# Get the profit/loss - this returns a list
pl = trading.betting.list_market_profit_and_loss(
    market_ids=[market_id], 
    include_bsp_bets='true', 
    include_settled_bets='true')
pl

In [ ]:
# # Create a profit/loss DataFrame
# pl_df = pd.DataFrame(pl[0]._data['profitAndLosses']).assign(marketId=pl[0].market_id)
# pl_df

In [ ]:
# Define a date filter - get all bets for the past 4 days
four_days_ago = (datetime.datetime.utcnow() - datetime.timedelta(days=4)).strftime("%Y-%m-%dT%TZ")
acct_statement_date_filter = betfairlightweight.filters.time_range(from_=four_days_ago)

# Request account statement
account_statement = trading.account.get_account_statement(item_date_range=acct_statement_date_filter)
account_statement

In [ ]:
# Create df of recent transactions
recent_transactions = pd.DataFrame(account_statement['accountStatement'])
recent_transactions
